In [68]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import timedelta

In [87]:
data = pd.read_csv("Results/NamedEntities.csv",encoding="utf-16")
data = data.drop("Unnamed: 0",axis=1)

In [88]:
data["Datetime"] = pd.to_datetime(data["Datetime"])
data.dtypes

Message ID             int64
Text                  object
Country               object
Datetime      datetime64[ns]
NEs                   object
dtype: object

In [89]:
def clean(nes):
    global count
    nes = nes[1:-1].split(",")
    result = []
    for ne in nes:
        n = ne.strip()
        if n == "\u200b\u200b-":
            continue
        elif n in result:
            continue
        else:
            result.append(n)
    
    result = ",".join(result)
    return result

In [90]:
data["NEs"] = data.apply(lambda x: clean(x["NEs"]),axis=1)

In [91]:
data

,Message ID,Text,Country,Datetime,NEs
0,1594,​​Упс.),Ukraine,2022-02-01 07:57:15,
1,1595,ТЕОРИЯ ХАОСА\n\nhttps://apeiron.school/chaos-t...,Ukraine,2022-02-01 18:08:01,ТЕОРИЯ ХАОСА\n\nhttps://apeiron.school/chaos-t...
2,1598,​​- Завтра начинается очередной модуль курса «...,Ukraine,2022-02-04 09:50:46,Завтра
3,1599,​​- Politeka с неотразимой Ингой Мезерей:\n\n0...,Ukraine,2022-02-04 13:54:08,"Politeka,Ингой Мезерей,Эрдогана,Украину,Украин..."
4,1600,​​- Она не спит. У неё - инсайт.)\n\nУченье - ...,Ukraine,2022-02-05 21:20:48,
...,...,...,...,...,...
220399,56079,"Текст предупреждений, которые полицейские прин...",Russia,2022-03-03 19:52:18,"Дождя,ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ),ВЫПОЛНЯЮЩИМ ..."
220400,56080,NaN,Russia,2022-03-03 19:52:18,
220401,56081,Смотрите фильм Веры Кричевской F@ck This Job в...,Russia,2022-03-05 16:14:44,"Веры Кричевской,ДАННОЕ СООБЩЕНИЕ (МАТЕРИАЛ),ВЫ..."
220402,56082,Важное заявление «Роскомсвободы». Проект проси...,Russia,2022-03-09 13:02:22,"Роскомсвободы,Запад,России,ДАННОЕ СООБЩЕНИЕ (М..."


In [134]:
def getWindow(date,df,country,window=2):
    switch = {"Ukraine":"Russia","Russia":"Ukraine"}
    country = switch[country]
    start_date = date - timedelta(days=window)
    end_date = date + timedelta(days=window)
    mask = (df['Datetime'] > start_date) & (df['Datetime'] <= end_date) & (df["Country"]==country)
    return df.loc[mask]

In [135]:
def overlap_coeff(nes1,nes2):
    nes1 = set(nes1.split(","))
    nes2 = set(nes2.split(","))
    overlap = nes1 & nes2
    return len(overlap) / max([len(nes1),len(nes2)])
    

In [136]:
def event_matching(nes,df_potential):
    top_score = 0
    winner = None
    for i,message in df_potential.iterrows():
        score = overlap_coeff(nes,message["NEs"])
        if score > top_score:
            winner = message
            top_score = score
    return winner,top_score

In [137]:
# Testing

test_m = data.iloc[12]
date = test_m["Datetime"]
test_nes = test_m["NEs"]
window = getWindow(date,data,"Ukraine")

matched,score = event_matching(test_nes,window)

print("Test:")
print(test_m["Text"])

print("NEs:",test_nes)

print()
print("Matched:")

print(matched["Text"])
print("Matched NEs:",matched["NEs"])
print("Score:",score)

Test:
​​- Мое небольшое интервью «Новинам.live” о международной обстановке вокруг Украины.

«Ближайшие две недели - решающие»:
 
https://novyny.live/politics/rossiia-doigralas-ei-dali-po-shchekam-blizhaishie-2-nedeli-reshaiushchie-interviu-alekseia-arestovicha-o-voine-donbasse-i-zapade-38876.html
NEs: Мое,Новинам.live,Украины

Matched:
Зеленский подтвердил готовность Украины выполнять Минские соглашения, заявил Макрон.

@rt_russian
Matched NEs: Зеленский,Украины,Макрон
Score: 0.3333333333333333
